In [45]:
#from fastapi import FastAPI
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn import tree
from datetime import date
from datetime import datetime
#from fastapi import Request, FastAPI
from sklearn.model_selection import train_test_split
#from typing import Any, Dict, AnyStr, List, Union
from pydantic import BaseModel
import json
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier

**Desarrollo:**

In [46]:
df = pd.read_excel("2. Datos para entrenar.xlsx",engine='openpyxl')
filtro=[]
for i,j in enumerate(df['fecha_nacimiento']):
    filter=type(df['fecha_nacimiento'][0])
    if type(j)==filter:
        filtro.append(True)
    else:
        filtro.append(False) 
df=df[filtro]
edad=[]
for k,h in enumerate(df['fecha_nacimiento']):
    edad.append(2022-h.year)
df['Edad']=edad

In [47]:
salidas1= df[['consulta_id','cups_solictud','descripcion','cantidad','prescripcion','tipo','codigo_medicamento','descripcion_medicamento','cantidad_medicamento','prescripcion_medicamento']]
entradas= df.drop(['diagnostico_secundario','diagnostico_terciario','cuarto_diagnostico','presentacion','fecha_nacimiento','cita_id','id_solicitud_enviada','medicamento_enviado',"TEST_HERRERA_Y_HURTADO",'cups_solictud','descripcion','cantidad','prescripcion','tipo','codigo_medicamento','descripcion_medicamento','cantidad_medicamento','prescripcion_medicamento'], axis=1)

In [48]:
lines=df['diagnostico_principal'].drop_duplicates().to_list()
with open('your_file.txt', 'w') as f:
    for line in lines:
        f.write(f"{line}\n")

In [49]:
salidas1=salidas1.fillna(0)
entradas=entradas.fillna(0)

In [50]:
cups_solictud=[]
descripcion=[]
cantidad=[]
prescripcion=[]
tipo=[]
codigo_medicamento=[]
descripcion_medicamento=[]
cantidad_medicamento=[]
prescripcion_medicamento=[]
#presentacion=[]
enes=[]
for n in salidas1['consulta_id'].drop_duplicates():
    if len(salidas1[salidas1['consulta_id']==n])>0:
        cups_solictud1=' '
        descripcion1=' '
        cantidad1=' '
        prescripcion1=' '
        tipo1=' '
        codigo_medicamento1=' '
        descripcion_medicamento1=''
        cantidad_medicamento1=''
        prescripcion_medicamento1=''
        #presentacion1=''
        for k in range(len(salidas1[salidas1['consulta_id']==n])):
            cups_solictud1=cups_solictud1+'-'+str(salidas1[salidas1['consulta_id']==n]['cups_solictud'].to_list()[k])
            descripcion1=descripcion1+'-'+str(salidas1[salidas1['consulta_id']==n]['descripcion'].to_list()[k])
            cantidad1=cantidad1+'-'+str(salidas1[salidas1['consulta_id']==n]['cantidad'].to_list()[k])
            prescripcion1=prescripcion1+'-'+str(salidas1[salidas1['consulta_id']==n]['prescripcion'].to_list()[k])
            tipo1=tipo1+'-'+str(salidas1[salidas1['consulta_id']==n]['tipo'].to_list()[k])
            codigo_medicamento1=codigo_medicamento1+'-'+str(salidas1[salidas1['consulta_id']==n]['codigo_medicamento'].to_list()[k])
            descripcion_medicamento1=descripcion_medicamento1+'-'+str(salidas1[salidas1['consulta_id']==n]['descripcion_medicamento'].to_list()[k])
            cantidad_medicamento1=cantidad_medicamento1+'-'+str(salidas1[salidas1['consulta_id']==n]['cantidad_medicamento'].to_list()[k])
            prescripcion_medicamento1=prescripcion_medicamento1+'-'+str(salidas1[salidas1['consulta_id']==n]['prescripcion_medicamento'].to_list()[k])
            #presentacion1=presentacion1+'-'+str(salidas1[salidas1['consulta_id']==n]['presentacion'].to_list()[k])
        cups_solictud.append(cups_solictud1)
        descripcion.append(descripcion1)
        cantidad.append(cantidad1)
        prescripcion.append(prescripcion1)
        tipo.append(tipo1)
        codigo_medicamento.append(codigo_medicamento1)
        descripcion_medicamento.append(descripcion_medicamento1)
        cantidad_medicamento.append(cantidad_medicamento1)
        prescripcion_medicamento.append(prescripcion_medicamento1)
        #presentacion.append(presentacion1)
        enes.append(n)
    else:
        continue

In [51]:
salidas = pd.DataFrame()
salidas['consulta_id']=enes
salidas['cups_solictud']=cups_solictud
salidas['descripcion']=descripcion
salidas['cantidad']=cantidad
salidas['prescripcion']=prescripcion
salidas['tipo']=tipo
salidas['codigo_medicamento']=codigo_medicamento
salidas['descripcion_medicamento']=descripcion_medicamento
salidas['cantidad_medicamento']=cantidad_medicamento
salidas['prescripcion_medicamento']=prescripcion_medicamento

In [52]:
#salidas['presentacion']=presentacion

In [53]:
for j in entradas['consulta_id'].drop_duplicates():
    if j in enes:
        continue
    else:
        entradas = entradas.drop(entradas[entradas[j]==True].index)
entradas['consulta_id']=entradas['consulta_id'].drop_duplicates().dropna()

In [54]:
entradas=entradas.dropna()

In [55]:
entradas=entradas.sort_values(by=['consulta_id'])
salidas=salidas.sort_values(by=['consulta_id'])

In [56]:
entradas=entradas.drop(columns=['consulta_id'])
salidas=salidas.drop(columns=['consulta_id'])

In [57]:
entradas['T_A'] = entradas['T_A'].str.replace('-','/')

In [58]:
t_a = entradas["T_A"].str.split('/', expand=True)
t_a.columns = ['T_A1', 'T_A2']
entradas = pd.concat([entradas, t_a], axis=1)

In [59]:
entradas['TEST_DE_FINDRISK']=entradas['TEST_DE_FINDRISK'].astype(str).apply(lambda x: x[:2]).str.replace(':',' ').astype(float)

In [60]:
entradas['TASA_DE_FILTRACION_GLOMERULAR']=entradas['TASA_DE_FILTRACION_GLOMERULAR'].astype(str).apply(lambda x: x[:2]).astype(float)

In [61]:
entradas['TEST_DE_FRAMINGHAM']=entradas['TEST_DE_FRAMINGHAM'].astype(str).apply(lambda x: x[:1]).str.replace('%',' ').astype(float)

In [62]:
salidas=salidas.fillna(0)
entradas=entradas.fillna(0)

In [63]:
entradas=entradas.drop(['T_A'], axis=1)

In [64]:
clf1=tree.DecisionTreeClassifier(criterion='gini',max_depth=30,ccp_alpha=0.05)
#clf3=tree.DecisionTreeClassifier(criterion='gini',max_depth=30,ccp_alpha=0.05)
tablas_enc = dict()
i=0
for j in entradas.columns.tolist():
    entradas[j]=entradas[j].fillna(0)
    if entradas[j].dtype=='O':
        entradas[j]=entradas[j].astype(str)
        tablas_enc[j] = OrdinalEncoder(categories=[entradas[j].unique().tolist()])
        tablas_enc[j].fit(entradas[[j]])
        entradas[j+'cat']= tablas_enc[j].transform(entradas[[j]])
    else:
        entradas[j+'cat'] = entradas[j]
        continue
for i in salidas.columns.tolist():
    salidas[i]=salidas[i].fillna(0)
    if salidas[i].dtype=='O':
        salidas[i]=salidas[i].astype(str)
        tablas_enc[i]=OrdinalEncoder(categories=[salidas[i].unique().tolist()])
        tablas_enc[i].fit(salidas[[i]])
        salidas[i+'cat'] = tablas_enc[i].transform(salidas[[i]])
    else:
        salidas[i+'cat'] = salidas[i]
        continue
salidas_cat=[]
for k in salidas.columns.tolist():
    if 'cat' in k:
        salidas_cat.append(k)
    else:
        continue
entradas_cat=[]
for h in entradas.columns.tolist():
    if 'cat' in h:
        entradas_cat.append(h)
    else:
        continue

In [65]:
salidas_cat1=[]
salidas_cat1.append('cups_solictudcat')
salidas_cat1.append('descripcioncat')
salidas_cat1.append('cantidadcat')
salidas_cat1.append('prescripcioncat')
salidas_cat1.append('tipocat')
salidas_cat2=[]
salidas_cat2.append('codigo_medicamentocat')
salidas_cat2.append('descripcion_medicamentocat')
salidas_cat2.append('cantidad_medicamentocat',)
salidas_cat2.append('prescripcion_medicamentocat')

In [66]:
#del clf1,clf2,clf,clf3

In [67]:
Y1=salidas[salidas_cat]
X1=entradas[entradas_cat]
#Y2=salidas[salidas_cat2]
#X2=salidas[salidas_cat1]
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, Y1, test_size=0.20)
#X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, Y2, test_size=0.20)

clf1=clf1.fit(X_train1, y_train1)
#clf2=clf3.fit(X_train2, y_train2)

In [68]:
#from joblib import dump, load
#dump(clf1, 'tree1c.joblib')
#dump(clf2, 'tree2c.joblib')

In [69]:
#dump(tablas_enc, 'tablesa1.joblib')

In [70]:
#tablas_enc1 = load('tablesa.joblib')

In [71]:
from sklearn.tree import plot_tree

In [72]:
#tree.plot_tree(clf)
#plt.savefig("Imagen1.jpg")

In [93]:
dato_en=[]
dato_en.append(int('35'))#edad
dato_en.append(tablas_enc['sexo'].fit_transform([['M']]))#sexo
dato_en.append(tablas_enc['grupo_sanguineo'].fit_transform([['O+']]))#grupo_sanguineo
dato_en.append(float('90'))#peso
dato_en.append(int('110'))#talla
dato_en.append(float('40'))#temp
dato_en.append(int('60'))#F_C
dato_en.append(int('50'))#F_R
dato_en.append(float('5'))#Test de frindrisk
dato_en.append(float('102'))#Tasa de filtracion
dato_en.append(float('0'))#Test de framingan ,porcentaje
dato_en.append(tablas_enc['diagnostico_principal'].fit_transform([['K021']]))#diagnostico_principal
#dato_en.append(tablas_enc['diagnostico_secundario'].fit_transform([['0']]))#diagnostico_secundario
#dato_en.append(tablas_enc['diagnostico_terciario'].fit_transform([['0']]))#diagnostico_terciario
#dato_en.append(tablas_enc['cuarto_diagnostico'].fit_transform([['0']]))#cuarto_diagnostico
dato_en.append(int('110'))#T_a1
dato_en.append(int('140'))#T_a2

In [94]:
prediccion= clf1.predict([dato_en])
lists = prediccion.tolist()
#prediccion2= clf2.predict(lists)
#lists2 = prediccion2.tolist()

In [95]:
respuesta2={
       "cups_solictud":tablas_enc['cups_solictud'].inverse_transform([[prediccion[0][0]]])[0][0],
       "descripcion" :tablas_enc['descripcion'].inverse_transform([[prediccion[0][1]]])[0][0],
       "cantidad":tablas_enc['cantidad'].inverse_transform([[prediccion[0][2]]])[0][0],
       "prescripcion": tablas_enc['prescripcion'].inverse_transform([[prediccion[0][3]]])[0][0][0][0],
       "tipo": tablas_enc['tipo'].inverse_transform([[prediccion[0][4]]])[0][0],
       "codigo_medicamento":tablas_enc['codigo_medicamento'].inverse_transform([[prediccion[0][0]]])[0][0],
       "descripcion_medicamento":tablas_enc['descripcion_medicamento'].inverse_transform([[prediccion[0][1]]])[0][0],
       "cantidad_medicamento":tablas_enc['cantidad_medicamento'].inverse_transform([[prediccion[0][2]]])[0][0],
       "prescripcion_medicamento":tablas_enc['prescripcion_medicamento'].inverse_transform([[prediccion[0][3]]])[0][0],
       #"presentacion":tablas_enc['presentacion'].inverse_transform([[prediccion2[0][4]]])[0][0]
    }
print(respuesta2)

{'cups_solictud': ' -902207.0', 'descripcion': ' -HEMOGRAMA I (HEMOGLOBINA HEMATOCRITO Y LEUCOGRAMA) MANUAL', 'cantidad': ' -1.0', 'prescripcion': ' ', 'tipo': ' -Laboratorios', 'codigo_medicamento': ' -CUM000171-CUM027213-CUM001309-CUM002522-CUM004094-CUM009446-CUM001279-CUM036626-CUM001279-CUM016670-CUM005190-CUM036626-CUM001279-CUM000187-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-CUM000171-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0', 'descripcion_medicamento': '-METOCARBAMOL TABLETAS 750MG-CALCITRIOL 0.5 MCG-TIAMINA TABLETAS 300 MG-NAPROXENO TABLETAS 500 MG-ACIDO FOLICO TABLETAS 1 MG.-OMEPRAZOL 40 MG-ACIDO ASCORBICO 500 MG. TAB

In [76]:
prueba=clf1.predict(X_test1)
prueba2=clf2.predict(X_test2)

In [77]:
prueba=prueba.tolist()
cups_solictud_prueba=[]
descripcion_prueba=[]
cantidad_prueba=[]
prescripcion_prueba=[]
tipo_prueba=[]
codigo_medicamento_prueba=[]
descripcion_medicamento_prueba=[]
cantidad_medicamento_prueba=[]
prescripcion_medicamento_prueba=[]
#presentacion_prueba=[]
for i in range(0,len(prueba)):
    cups_solictud_prueba.append(prueba[i][0])
    descripcion_prueba.append(prueba[i][1])
    cantidad_prueba.append(prueba[i][2])
    prescripcion_prueba.append(prueba[i][3])
    tipo_prueba.append(prueba[i][4])
    codigo_medicamento_prueba.append(prueba2[i][0])
    descripcion_medicamento_prueba.append(prueba2[i][1])
    cantidad_medicamento_prueba.append(prueba2[i][2])
    prescripcion_medicamento_prueba.append(prueba2[i][3])
    #presentacion_prueba.append(prueba[i][9])

In [78]:
#(y_test['presentacioncat']==presentacion_prueba).value_counts()

In [79]:
(y_test2['prescripcion_medicamentocat']==prescripcion_medicamento_prueba).value_counts()

False    77
True     11
Name: prescripcion_medicamentocat, dtype: int64

In [80]:
(y_test2['cantidad_medicamentocat']==cantidad_medicamento_prueba).value_counts()

False    85
True      3
Name: cantidad_medicamentocat, dtype: int64

In [81]:
(y_test2['descripcion_medicamentocat']==descripcion_medicamento_prueba).value_counts()

False    73
True     15
Name: descripcion_medicamentocat, dtype: int64

In [82]:
(y_test2['codigo_medicamentocat']==codigo_medicamento_prueba).value_counts()

False    77
True     11
Name: codigo_medicamentocat, dtype: int64

In [83]:
(y_test1['tipocat']==tipo_prueba).value_counts()

True     47
False    41
Name: tipocat, dtype: int64

In [84]:
(y_test1['prescripcioncat']==prescripcion_prueba).value_counts()

True     45
False    43
Name: prescripcioncat, dtype: int64

In [85]:
(y_test1['cantidadcat']==cantidad_prueba).value_counts()

True     47
False    41
Name: cantidadcat, dtype: int64

In [86]:
(y_test1['descripcioncat']==descripcion_prueba).value_counts()

False    81
True      7
Name: descripcioncat, dtype: int64

In [87]:
(y_test1['cups_solictudcat']==cups_solictud_prueba).value_counts()

False    81
True      7
Name: cups_solictudcat, dtype: int64

In [88]:
import tensorflow as tf
import numpy as np

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
oculta1 = tf.keras.layers.Dense(units=3, input_shape=[1])
oculta2 = tf.keras.layers.Dense(units=3)
salida_tensor = tf.keras.layers.Dense(units=1)
modelo_tensor = tf.keras.Sequential([oculta1, oculta2, salida_tensor])

In [ ]:
modelo_tensor.compile(
    optimizer=tf.keras.optimizers.Adam(0.1),
    loss='mean_squared_error'
)